# Intro

In this project, we'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. We'll be using this dataset from Oxford of 102 flower categories

The project is broken down into multiple steps:

Load the image dataset and create a pipeline.
Build and Train an image classifier on this dataset.
Use your trained model to perform inference on flower images.
We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here the network will be learning about flowers and end up as a command line application. 

In [1]:
# The new version of dataset is only available in the tfds-nightly package.
%pip --no-cache-dir install tfds-nightly --user
# DON'T MISS TO RESTART THE KERNEL


     |████████████████████████████████| 4.0 MB 7.1 MB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
# Import TensorFlow 
import tensorflow as tf
!python -m tensorflow_datasets.scripts.download_and_prepare --register_checksums=True --datasets=oxford_flowers102
import tensorflow_datasets as tfds
import tensorflow_hub as hub

In [ ]:
# Make all other necessary imports.
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import time
import numpy as np
import pandas as pd

Load the Dataset
Here we'll use tensorflow_datasets to load the Oxford Flowers 102 dataset. This dataset has 3 splits: 'train', 'test', and 'validation'. we'll also need to make sure the training data is normalized and resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet, but we'll still need to normalize and resize the images to the appropriate size.

In [ ]:
# Download data to default local directory "~/tensorflow_datasets"
!python -m tensorflow_datasets.scripts.download_and_prepare --register_checksums=True --datasets=oxford_flowers102

#Loads the dataset with TensorFlow Datasets
dataset, dataset_info = tfds.load('oxford_flowers102', as_supervised=True, with_info=True)

# Create a training set, a validation set and a test set.
training_set, test_set , validation_set = dataset['train'], dataset['test'] , dataset['validation']


In [ ]:
# Prints the shape and corresponding label of 3 images in the training set.

shape_images = dataset_info.features['image'].shape

for image, label in training_set.take(3):
    image = image.numpy().squeeze()
    label = label.numpy()

    plt.imshow(image, cmap= plt.cm.binary)
    plt.colorbar()
    plt.show()
    print('The shape of the image is ', shape_images )
    print('The label of this image is:', label)


In [ ]:
# Gets the number of examples in each set from the dataset info.
print('Training set has {:,} elements '.format(dataset_info.splits['train'].num_examples))
print('Testing set has {:,} elements '.format(dataset_info.splits['test'].num_examples))
print('Validation set has {:,} elements '.format(dataset_info.splits['validation'].num_examples))
# Gets the number of classes in the dataset from the dataset info.
num_classes = dataset_info.features['label'].num_classes
print('Dataset has {:,} Classes'.format(num_classes))

In [ ]:
# Plots 1 image from the training set. 
for image, label in training_set.take(1):
    image = image.numpy().squeeze()
    label = label.numpy()

plt.imshow(image, cmap=plt.cm.binary)
plt.title(label)
plt.colorbar()
plt.show()

# Label Mapping
We'll also need to load in a mapping from label to category name. This will be in the file label_map.json. It's a JSON object which you can read in with the json module. This will give a dictionary mapping the integer coded labels to the actual names of the flowers.

In [ ]:
import json
with open('label_map.json', 'r') as f:
    class_names = json.load(f)


In [ ]:
# Plots 1 image from the training set.  
for image, label in training_set.take(1):
    image = image.numpy().squeeze()
    label = label.numpy()

plt.imshow(image, cmap=plt.cm.binary)
plt.title(class_names[format(label)])
plt.colorbar()
plt.show()

#Create Pipeline

In [ ]:
# Create a pipeline for each set.
batch_size = 32
image_size = 224

num_training_examples = 1020

def format_image(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (image_size, image_size))
    image /= 255
    return image, label


training_batches = training_set.shuffle(num_training_examples//4).map(format_image).batch(batch_size).prefetch(1)
validation_batches = validation_set.map(format_image).batch(batch_size).prefetch(1)
testing_batches = test_set.map(format_image).batch(batch_size).prefetch(1)

# Build and Train the Classifier
Now that the data is ready, we're ready to build and train the classifier. We'll use the MobileNet pre-trained model from TensorFlow Hub to get the image features. 


In [ ]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(URL, input_shape=(image_size, image_size,3))
feature_extractor.trainable = False


model = tf.keras.Sequential([
        feature_extractor,
        tf.keras.layers.Dense(102, activation = 'softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

EPOCHS = 10

history = model.fit(training_batches,
                    epochs=EPOCHS,
                    validation_data=validation_batches)

In [ ]:
# Plots the loss and accuracy values achieved during training for the training and validation set.

training_accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']

training_loss = history.history['loss']
validation_loss = history.history['val_loss']

epochs_range=range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, training_accuracy, label='Training Accuracy')
plt.plot(epochs_range, validation_accuracy, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, training_loss, label='Training Loss')
plt.plot(epochs_range, validation_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

#Testing the Network

In [ ]:
# TODO: Print the loss and accuracy values achieved on the entire test set.

loss, accuracy = model.evaluate(testing_batches)

print('\nLoss on the TEST Set: {:,.3f}'.format(loss))
print('Accuracy on the TEST Set: {:.3%}'.format(accuracy))


#Saving the Model

In [ ]:
t = time.time()

saved_keras_model_filepath = './{}.h5'.format(int(t))

model.save(saved_keras_model_filepath)


In [ ]:
# Recovers the model that was previously saved
reloaded_keras_model = tf.keras.models.load_model(saved_keras_model_filepath, custom_objects={'KerasLayer':hub.KerasLayer})

#Inference for Classification
Now we'll write a function that uses your trained network for inference. The function called predict takes an image, a model, and then returns the top  K  most likely class labels along with the probabilities. 

The predict function will also need to handle pre-processing the input image such that it can be used by the model. 


In [ ]:
def process_image(image):
    image = tf.convert_to_tensor(image)

    image = tf.image.resize(image, size = (224,224))
    image /= 255
    image = image.numpy()
    return image

In [ ]:
from PIL import Image

image_path = './test_images/hard-leaved_pocket_orchid.jpg'
im = Image.open(image_path)
test_image = np.asarray(im)

processed_test_image = process_image(test_image)

fig, (ax1, ax2) = plt.subplots(figsize=(10,10), ncols=2)
ax1.imshow(test_image)
ax1.set_title('Original Image')
ax2.imshow(processed_test_image)
ax2.set_title('Processed Image')
plt.tight_layout()
plt.show()

In [ ]:
#Creates the predict function

def predict(image_path, model, top_k=5):
    image = Image.open(image_path)
    image = np.asarray(image)
    image = process_image(image)
    image = np.expand_dims(image, axis = 0)
    predictions = model.predict(image)
    
    top_ps, top_k_indices = tf.math.top_k(predictions, top_k)

    
    return top_ps.numpy()[0], top_k_indices.numpy()[0]

In [ ]:
# Plot the input image along with the top 5 classes
image_path = './test_images/cautleya_spicata.jpg'

probabilities, classes = predict(image_path,model,5)

im = Image.open(image_path)
test_image = np.asarray(im)

processed_test_image = process_image(test_image)



top_names = []
for i in classes:
    top_names.append(class_names[str(i+1)])
    
fig, (ax1, ax2) = plt.subplots(figsize=(10,10), ncols=2)
ax1.imshow(processed_test_image)
ax1.set_title(top_names[0],{'fontsize':20})
ax2.barh(np.arange(5), probabilities)
ax2.set_aspect(0.1)
ax2.set_yticks(np.arange(5))
ax2.set_yticklabels(top_names, size='small');
ax2.set_title('Class Probability',{'fontsize':20})
ax2.set_xlim(0, 1.1)
plt.tight_layout()
plt.show()